In [9]:
import pandas as pd
from IPython.display import Markdown

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.model import build_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")

## Model construction

In [2]:
param_info = load_param_info()
params = param_info["value"]

In [ ]:
model, desc = build_model(params)
Markdown(desc)

## Parameters

In [4]:
test_params = {
	'contact_rate': 9.543921666073745,
	'self_recovery_rate': 0.29688347664390236,
	'screening_scaleup_shape': 0.19889392155636598,
	'screening_inflection_time': 2009.3016735363497,
	'time_to_screening_end_asymp': 0.3361714758423252,
	'rr_infection_latent': 0.116003030825429,
	'rr_infection_recovered': 0.8720080623263372,
	'seed_time': 1881.49881702568,
	'seed_duration': 19.549815951426623,
	'seed_rate': 70.64648160388384,
	'base_sensitivity': 0.2867398245514573,
	'genexpert_sensitivity': 0.826222381252117,
	'progression_multiplier': 1.436023746975478,
}


In [5]:
model.run(params)

In [ ]:
param_text = "The parameters used in this model are listed below."
Markdown(param_text)

In [ ]:
fixed_param_table = get_param_table(param_info)
Markdown(fixed_param_table.to_markdown())

## Results

In [12]:
targets = load_targets()
outs = model.get_derived_outputs_df()
indicator_names = const.indicator_names
compartments = const.compartments

We used 2000 - 2022 notification data obtained from Global TB Report [@whotb2023] and prevalence data from 2013 - 2014 Indonesia TB Prevalence Survey [@indoprevsurv2015] as the calibration and optimisation target data.

### Size of each compartment

In [ ]:
fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
fig.add_scatter(x=targets["census_pop"].index, y=targets["census_pop"], mode="markers", name="Census data")
set_plot_label(fig, indicator_names, "Compartment size")
display_plot(fig, "comp_size", "svg")
#fig

### TB Notifications

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["notif"],
                     "notification",
                     "Notification",
                     "Modelled vs Data",
                     "Actual data (All periods)")
#fig.add_scatter(x=targets["new_cases"].index, y=targets["new_cases"], mode="markers", name="NTCP data")
fig.add_scatter(x=targets["notif2000"].index, y=targets["notif2000"], mode="markers", name="Target data")
fig.update_xaxes(range=[1978, 2023])
display_plot(fig, "notification", "svg")
#fig

### TB incidence, prevalence, and deaths

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["prevalence"],
                     "prevalence",
                     "Prevalence",
                     "Modelled vs Data (WHO)",
                     "Actual data")

display_plot(fig, "prevalence", "svg")

In [ ]:
fig = outs[["incidence", "prevalence", "mortality"]].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "inc_prev_mort", "svg")

### Notification per incidence

In [ ]:
actual_data = targets["notif"] / targets["incidence_num"] * 100
fig = plot_model_vs_actual(outs, 
                     actual_data,
                     "notification_per_incidence",
                     "Notification per incidence",
                     "Modelled vs Data",
                     "Actual data")

display_plot(fig, "notification_per_incidence", "svg")

### Percentage of latent TB infection

We calculated the percentage of people with latent TB by dividing the sum of people in the `early_latent` and `late_latent` compartments by the total population.

In [ ]:
fig = outs["percentage_latent"].plot()
set_plot_label(fig, indicator_names, "value")
display_plot(fig, "percentage_latent", "svg")

## References